In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
import os
for dirname, _, filenames in os.walk('/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
data = pd.read_csv('train_clear.csv')
# Keeping only the neccessary columns
data = data[['text_clear','emotion']]

In [4]:
data

,text_clear,emotion
0,people post add snapchat dehydrated cuz man thats,anticipation
1,trump dangerous freepress world trumplegacy cnn,sadness
2,issa stalk tasha 😂😂😂,fear
3,thx best time tonight story heartbreakingly au...,joy
4,wait supply liscus,anticipation
...,...,...
1455558,im happy nowonder happy 👏👏👏👏👏,joy
1455559,circumtance id like thankful almighty jesus ch...,joy
1455560,there currently girls walk library handing red...,joy
1455561,ah corporate life date relative anachronism jo...,joy


In [16]:
test = pd.read_csv('test_clear.csv')
# Keeping only the neccessary columns
t_data = test[['text_clear']]

In [18]:
t_data["text_clear"].fillna(".", inplace = True)

C:\Users\oscar\AppData\Local\Temp\ipykernel_16732\3177639989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_data["text_clear"].fillna(".", inplace = True)


In [5]:
data["text_clear"].fillna(".", inplace = True)

In [6]:
max_fatures = 50000
tokenizer = Tokenizer(num_words=max_fatures, split=' ',filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')
tokenizer.fit_on_texts(data['text_clear'].values)
X = tokenizer.texts_to_sequences(data['text_clear'].values)

In [7]:
X = pad_sequences(X)

In [8]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 36, 128)           6400000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 36, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 8)                 1576      
Total params: 6,656,376
Trainable params: 6,656,376
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
Y = pd.get_dummies(data['emotion']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(975227, 36) (975227, 8)
(480336, 36) (480336, 8)


In [13]:
iY = pd.get_dummies(data['emotion'])

In [14]:
iY

,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1455558,0,0,0,0,1,0,0,0
1455559,0,0,0,0,1,0,0,0
1455560,0,0,0,0,1,0,0,0
1455561,0,0,0,0,1,0,0,0


In [12]:
Y

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [15]:
label=['anger','anticipation','disgust','fear','joy','sadness','surprise','trust']

In [44]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
30476/30476 - 8658s - loss: 1.1908 - accuracy: 0.5746
Epoch 2/7
30476/30476 - 8635s - loss: 1.1684 - accuracy: 0.5823
Epoch 3/7
30476/30476 - 8646s - loss: 1.1529 - accuracy: 0.5884
Epoch 4/7
30476/30476 - 8638s - loss: 1.1412 - accuracy: 0.5932
Epoch 5/7
30476/30476 - 8660s - loss: 1.1322 - accuracy: 0.5964
Epoch 6/7
30476/30476 - 8868s - loss: 1.1243 - accuracy: 0.5996
Epoch 7/7
30476/30476 - 8721s - loss: 1.1180 - accuracy: 0.6023


In [11]:
validation_size = 150000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

20646/20646 - 461s - loss: 1.2466 - accuracy: 0.5512
score: 1.25
acc: 0.55


In [45]:
tX = tokenizer.texts_to_sequences(t_data['text_clear'].values)

In [46]:
tX = pad_sequences(tX,maxlen=36)

In [37]:
tX.shape()

TypeError: 'tuple' object is not callable

In [47]:
ans=[]
for x in range(len(tX)):
    
    result = model.(tX[x].reshape(1,tX.shape[1]),batch_size=1,verbose = 0)[0]
    
    la=label[np.argmax(result)]
    ans.append(la)
    
    '''
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1
    '''

KeyboardInterrupt: 

In [ ]:
import time
ans=[]
clock = time.time()
for x in range(len(tX)):
    
    result = model(tX[x].reshape(1,tX.shape[1]),training = False)
    res = np.array(result)
    
    la=label[np.argmax(res)]
    ans.append(la)
    
    '''
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1
    '''
print(time.time() - clock,'s')

In [39]:
test['emtion']=ans


In [40]:
test

,tweet_id,text,identification,text_clear,emtion
0,0x28b412,"Confident of your obedience, I write to you, k...",test,confident obedience write know ask philemon bi...,anticipation
1,0x2de201,"""Trust is not the same as faith. A friend is s...",test,trust faith friend trust put faith mistake chr...,anticipation
2,0x218443,When do you have enough ? When are you satisfi...,test,satisfied goal money materialism money possession,joy
3,0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test,god wake chase day godsplan godswork,anticipation
4,0x26289a,"In these tough times, who do YOU turn to as yo...",test,tough time turn symbol hope,trust
...,...,...,...,...,...
411967,0x2913b4,"""For this is the message that ye heard from th...",test,message ye heard begin love john kjv,anticipation
411968,0x2a980e,"""There is a lad here, which hath five barley l...",test,lad hath barley loaves small fish john,anticipation
411969,0x316b80,When you buy the last 2 tickets remaining for ...,test,buy ticket remain sell mixedfeeling butimthatp...,sadness
411970,0x29d0cb,I swear all this hard work gone pay off one da...,test,swear hard work go pay day😈💰💸,joy


In [41]:
to_sub=test.drop(['text','identification','text_clear'],axis=1)
to_sub.columns = ['id','emotion']
to_sub.to_csv('submission_21.csv',index=0)

In [42]:
to_sub

,id,emotion
0,0x28b412,anticipation
1,0x2de201,anticipation
2,0x218443,joy
3,0x2939d5,anticipation
4,0x26289a,trust
...,...,...
411967,0x2913b4,anticipation
411968,0x2a980e,anticipation
411969,0x316b80,sadness
411970,0x29d0cb,joy


In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")